Cell 1: Kết nối Google Drive

In [ ]:
# Cell 1: Mount Google Drive (để lưu file JSON, output model)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cell 2: Tạo Datase gốc và train với biến thể của model BERT

Cell 2.1: Thu thập dữ liệu gốc và tạo Dataset tương ứng

In [ ]:
<Hướng dẫn sau>

Cell 3: Cài Đặt Thư Viện

In [ ]:
# Cell 3: Cài đặt các thư viện Transformers, Datasets, Accelerate

!pip install transformers datasets accelerate

import torch
print("PyTorch version:", torch.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cell 4: (Tùy chọn) Đăng Nhập Hugging Face

In [ ]:
# Cell 4: Chỉ chạy nếu bạn muốn push model
from huggingface_hub import notebook_login
notebook_login()

Cell 5: Tải Model DeepSeek 1.5B

In [ ]:
# Cell 5: Load DeepSeek-R1-Distill-Qwen-1.5B

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("Hoàn tất load model và tokenizer!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Hoàn tất load model và tokenizer!


Cell 6: Chia Tập & Tiền Xử Lý

In [ ]:
# Cell 6: Tách train/val, rồi map dữ liệu

from datasets import load_dataset

data_path = "/content/drive/MyDrive/Project_CongVan_Classification/data/train_congvan_1000.json"
# Hãy sửa thành:
all_data = load_dataset("json", data_files={"dataset": data_path})["dataset"]
print("Tổng số mẫu công văn:", len(all_data))

# Chia 80% train, 20% val
split_data = all_data.train_test_split(test_size=0.2, seed=42)
train_data = split_data["train"]
val_data   = split_data["test"]
print("Số mẫu train =", len(train_data), ", Số mẫu val =", len(val_data))

def preprocess_function(example):
    prompt = f"Nội dung công văn: {example['input']}\nHướng xử lý:"
    full_text = prompt + " " + example['output']

    tokenized = tokenizer(
        full_text,
        padding="max_length",
        max_length=256,  # Giảm nếu OOM
        truncation=True
    )

    # Mask prompt
    prompt_tokenized = tokenizer(prompt, add_special_tokens=False)
    label_ids = tokenized["input_ids"].copy()

    prompt_len = len(prompt_tokenized["input_ids"])
    for i in range(prompt_len):
        if i < len(label_ids):
            label_ids[i] = -100

    tokenized["labels"] = label_ids
    return tokenized

train_dataset = train_data.map(preprocess_function, batched=False)
val_dataset   = val_data.map(preprocess_function,   batched=False)

# Kiểm tra 1 sample
print("\nTrain sample [0]:", train_dataset[0])

decoded_text = tokenizer.decode(train_dataset[0]["input_ids"])
print("Decoded token =>\n", decoded_text)

Generating dataset split: 0 examples [00:00, ? examples/s]

Tổng số mẫu công văn: 1000
Số mẫu train = 800 , Số mẫu val = 200


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]


Train sample [0]: {'input': 'Công văn số 803/912 ngày 03/10/2024: Phòng ban đề nghị xử lý công việc hành chính để đảm bảo đảm bảo sự minh bạch trong hoạt động để cải thiện mối quan hệ đối tác.', 'output': 'Giao cho phòng Marketing', 'input_ids': [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 

Cell 7: Thiết Lập TrainingArguments & Trainer

In [ ]:
# Cell 7: TrainingArguments + Trainer

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Project_CongVan_Classification/output-model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    logging_steps=50,
    save_steps=200,
    evaluation_strategy="steps",
    eval_steps=200,
    fp16=True,
    # gradient_checkpointing=True,  # bật nếu vẫn OOM
    report_to="none"
)

print("TrainingArguments đã sẵn sàng!")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)
print("Trainer đã được tạo thành công!")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-2ec7193a151b>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


TrainingArguments đã sẵn sàng!
Trainer đã được tạo thành công!


Cell 8: Huấn Luyện

In [ ]:
# Cell 8: Gọi trainer.train()

train_result = trainer.train()
print("Kết quả TrainOutput:", train_result)

# Lưu checkpoint cuối
trainer.save_model("/content/drive/MyDrive/Project_CongVan_Classification/final_model")

Step,Training Loss,Validation Loss
200,0.185000,0.187616
400,0.171600,0.171673
600,0.165500,0.167993
800,0.163900,0.166900
1000,0.164100,0.163906
1200,0.160300,0.162128


Kết quả TrainOutput: TrainOutput(global_step=1200, training_loss=0.22648831288019816, metrics={'train_runtime': 880.9738, 'train_samples_per_second': 2.724, 'train_steps_per_second': 1.362, 'total_flos': 5690748410265600.0, 'train_loss': 0.22648831288019816, 'epoch': 3.0})


Cell 9: Kiểm thử mô hình bằng input từ người dùng

In [ ]:
# Cell 9: Tương tác với người dùng để nhập nội dung công văn

from transformers import pipeline, AutoModelForCausalLM

# Load lại mô hình đã fine-tuned
model_finetuned = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Project_CongVan_Classification/final_model")

pipeline_infer = pipeline(
    "text-generation",
    model=model_finetuned,
    tokenizer=tokenizer,
    max_length=80,
    do_sample=False  # Giữ False để mô hình sinh kết quả deterministic (ko random)
)

print("Nhập nội dung công văn bạn muốn kiểm thử (hoặc gõ 'exit' để dừng).")

while True:
    user_input = input("\nNội dung công văn: ")
    if user_input.strip().lower() == "exit":
        print("Đã thoát thử nghiệm.")
        break

    # Tạo prompt
    prompt = f"Nội dung công văn: {user_input}\nHướng xử lý:"

    # Gọi pipeline để sinh kết quả
    result = pipeline_infer(prompt)
    output_text = result[0]["generated_text"]

    print("\n## Kết quả inference:")
    print(output_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Nhập nội dung công văn bạn muốn kiểm thử (hoặc gõ 'exit' để dừng).

Nội dung công văn: Văn bản quy định về việc thu chi trong văn phòng trường

## Kết quả inference:
Nội dung công văn: Văn bản quy định về việc thu chi trong văn phòng trường
Hướng xử lý: Giao cho phòng Pháp chế nhằm nâng cao hiệu quả công việc để cải thiện mối quan hệ đối tác. Yêu cầu phối hợp chặt chẽ giữa các phòng ban.
Hướng xử lý: Giao cho phòng Pháp chế để thực hiện đảm bảo sự min

Nội dung công văn: Lập quy trình tuyển dụng thêm sinh viên học ngành ngôn ngữ anh

## Kết quả inference:
Nội dung công văn: Lập quy trình tuyển dụng thêm sinh viên học ngành ngôn ngữ anh
Hướng xử lý: Giao cho phòng Đào tạo để đảm bảo tối ưu hóa hoạt động nội bộ để cải thiện mối quan hệ đối tác. Yêu cầu phối hợp chặt chẽ giữa các phòng ban.
Hướng xử lý: Giao cho phòng Đào tạo

Nội dung công văn: Xây dựng các khóa học máy tính

## Kết quả inference:
Nội dung công văn: Xây dựng các khóa học máy tính
Hướng xử lý: Giao cho phòng Đào tạo để đả